GERNERATE AND MODIFY (example of contex management)

In [ ]:
import os
import asyncio
from agents import Agent, Runner, RunContextWrapper, function_tool , TResponseInputItem
from agents.extensions.models.litellm_model import LitellmModel
from dataclasses import dataclass
from dotenv import load_dotenv

# ✅ Define context data structure
load_dotenv()

# Model info
MODEL = "gemini/gemini-1.5-flash"
gemini_api_key = os.getenv("GEMINI_API_KEY")

if not gemini_api_key:
    raise ValueError("❌ GEMINI_API_KEY not found in .env file!")
@dataclass
class UserInfo:
    name: str
    favorite_color: list[str]

# ✅ Define a tool that uses context

@function_tool
async def get_name(wrapper: RunContextWrapper[UserInfo]) -> str:
    return wrapper.context.name

@function_tool
async def change_name(wrapper: RunContextWrapper[UserInfo] , name: str) -> str:
    change_name_of_user = wrapper.context.name = name
    return change_name_of_user

@function_tool
async def greet_user(wrapper: RunContextWrapper[UserInfo]) -> str:
    user = wrapper.context
    return f"Hello {user.name}! Your favorite color is {user.favorite_color}."

@function_tool
async def get_favourite_colors(wrapper: RunContextWrapper[UserInfo]) -> list[str]:
    print("Getting favourite_colors")
    return wrapper.context.favorite_color

@function_tool
async def change_color(wrapper: RunContextWrapper[UserInfo], colors: list[str]) -> None:
    print("Adding colors to favourite_color")
    wrapper.context.favorite_color.extend(colors)





# async def main():
    # user_info = UserInfo(name="Ali", favorite_color=["blue"])  # context

agent = Agent[UserInfo](
    name="greeting_bot ,Color_telling bot, color_modifying bot",
    instructions=" if user ask for his name then give his name and if user ask to modify his name so do it as wellif the user ask you to greet me then call the (gree_user) fuction  and if user ask you to modify the fovourite colors then first get the colors by call (get_favourite_color) function and then modify it via call (change_color) fuction",
    model=LitellmModel(model=MODEL,api_key=gemini_api_key),
    tools=[greet_user,get_name,change_name,get_favourite_colors,change_color],
)
user_info = UserInfo(name="Ali", favorite_color=["blue"])
print("You are now chatting with the shopping assistant. Type 'exit' to end the conversation.")
convo_items: list[TResponseInputItem] = []

while True:
    print(f"your name is {user_info.name}")
    user_input = input("You: ")

    if user_input == "exit":
        print("Goodbye!")
        break
    
    convo_items.append({"content": user_input, "role": "user"})
    result = await Runner.run(agent,convo_items, context=user_info)
    

    print(f"Shopping Assistant: {result.final_output}")
    
    convo_items = result.to_input_list()